In [1]:
# GPU 연결 확인
import tensorflow as tf 
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices() )

2023-12-21 11:17:03.543051: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-21 11:17:03.607867: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 11:17:03.607888: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 11:17:03.608457: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 11:17:03.612255: I tensorflow/core/platform/cpu_feature_guar

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5794736580325347199
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 23156555776
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10276702926647980689
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2023-12-21 11:17:04.537645: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-21 11:17:04.668820: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-21 11:17:04.668941: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [7]:
import pandas as pd
import numpy as np
from PIL import Image
import ast
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import tensorflow_addons as tfa
import math

/home/myeong/.local/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [8]:
home_office_desks_df = pd.read_pickle("home_office_desks_df.pkl")

In [9]:
home_office_desks_df.shape

(3022, 9)

In [10]:
home_office_desks_df.head()

,Category,Title,Price,Product_Info,Product_Feature,Product_Text,Img_URL,Product URL,img_path
1,"[Home & Kitchen, Furniture, Home Office Furnit...",A AIRLLEN Computer Desk 63 Inch Modern Simple ...,$169.99,"{'Brand': 'A AIRLLEN', 'Product Dimensions': '...","{'Shape': 'Rectangular', 'Desk design': 'Compu...",【Actual Table Dimension】63 L x 23.62 D x 28.35...,https://m.media-amazon.com/images/I/71I9DrrwH9...,https://www.amazon.com/AIRLLEN-Computer-Workst...,/home/all/imgs/Desks/AAIRLLENComputerDeskInchM...
2,"[Home & Kitchen, Furniture, Home Office Furnit...",Cubiker Computer Home Office Desk with Drawers...,$79.99,"{'Brand': 'Cubiker', 'Product Dimensions': '23...","{'Shape': 'Rectangular', 'Desk design': 'Compu...",Modern Confident Style: Cubiker office compute...,https://m.media-amazon.com/images/I/81tfZZqqGd...,https://www.amazon.com/Cubiker-Computer-Office...,/home/all/imgs/Desks/CubikerComputerHomeOffice...
3,"[Home & Kitchen, Furniture, Home Office Furnit...",Rise UP Dual Motor Electric Standing Desk 60x3...,$489.99,"{'Brand': 'Uncaged Ergonomics', 'Product Dimen...","{'Shape': 'Rectangular', 'Desk design': 'Compu...","A beautiful, durable, dual-motor electric adju...",https://m.media-amazon.com/images/I/71Weo55ETw...,https://www.amazon.com/Electric-Adjustable-Erg...,/home/all/imgs/Desks/RiseUPDualMotorElectricSt...
4,"[Home & Kitchen, Furniture, Home Office Furnit...","CubiCubi Computer Desk, 40 inch Home Office De...",$54.99,"{'Brand': 'CubiCubi', 'Product Dimensions': '1...","{'Shape': 'Rectangular', 'Desk design': 'Compu...",Modern Simple Style: This computer desk suits ...,https://m.media-amazon.com/images/I/71TfWerRrA...,https://www.amazon.com/Cubicubi-Computer-Offic...,/home/all/imgs/Desks/CubiCubiComputerDeskinchH...
6,"[Home & Kitchen, Furniture, Home Office Furnit...","Furologee 66” L Shaped Desk with Power Outlet,...",$179.99,"{'Brand': 'Furologee', 'Product Dimensions': '...","{'Shape': 'T-Shape', 'Desk design': 'Computer ...",Desk with Charging Station: Equipped with 3 po...,https://m.media-amazon.com/images/I/71FlutIlti...,https://www.amazon.com/Furologee-Reversible-Co...,/home/all/imgs/Desks/FurologeeLShapedDeskwithP...


In [11]:
home_office_desks_df.columns

Index(['Category', 'Title', 'Price', 'Product_Info', 'Product_Feature',
       'Product_Text', 'Img_URL', 'Product URL', 'img_path'],
      dtype='object')

In [12]:
# 스타일 목록
styles = ['modern', 'contemporary', 'rustic', 'vintage', 'industrial', 'traditional', 'minimalist', 'scandinavian', 'classic', 'retro', 'art deco', 'mid-century modern', 'glam', 'coastal', 'shaker', 'mission', 'eclectic']

# 스타일 레이블 추출 함수
def extract_style(info):
    try:
        product_info = ast.literal_eval(info)
        style = product_info.get('Style', '').lower()
        return styles.index(style) if style in styles else -1
    except ValueError:
        return -1

# 이미지 처리 함수
def process_image(image_path):
    try:
        with Image.open(image_path) as img:
            # RGB로 변환하여 일관된 채널 수 보장
            if img.mode != 'RGB':
                img = img.convert('RGB')
            img_resized = img.resize((256, 256))
            img_array = np.array(img_resized) / 255.0  # 정규화
            return img_array
    except (IOError, OSError):
        return np.zeros((256, 256, 3))  # 빈 이미지 반환

# 스타일 레이블 및 이미지 데이터 추출
image_data = []
labels = []
for _, row in home_office_desks_df.iterrows():
    style_label = extract_style(row['Product_Info'])
    if style_label != -1:
        img_array = process_image(row['img_path'])
        image_data.append(img_array)
        labels.append(style_label)

# NumPy 배열 변환
image_data = np.array(image_data)
labels = to_categorical(labels, num_classes=len(styles))

# 데이터 분할
X_train, X_val_test, y_train, y_val_test = train_test_split(
    image_data, labels, test_size=0.2, random_state=42
)

# Secondary split: Split the 20% equally into validation and test sets (each 10% of the total data)
X_val, X_test, y_val, y_test = train_test_split(
    X_val_test, y_val_test, test_size=0.5, random_state=42
)

In [20]:
# EfficientNet 모델 구성
input_channels = 3
se_ratio = 4
expand_ratio = 6
width_coefficient = 1.0
depth_coefficient = 1.0
default_resolution = 256
depth_divisor= 8 
dropout_rate = 0.2
drop_connect_rate = 0.2

kernel_size = [3,3,5,3,5,5,3]
num_repeat = [1,2,2,3,3,4,1]
output_filters = [16,24,40,80,112,192,320]
strides = [1,2,2,2,1,2,1]
MBConvBlock_1_True  =  [True,False,False,False,False,False,False]

def round_repeats(repeats, depth_coefficient):
    return int(math.ceil(depth_coefficient * repeats))

def round_filters(filters, width_coefficient, depth_divisor):
    filters *= width_coefficient
    new_filters = int(filters + depth_divisor / 2) // depth_divisor * depth_divisor
    new_filters = max(depth_divisor, new_filters)
    if new_filters < 0.9 * filters:
        new_filters += depth_divisor
    return int(new_filters)

class DropConnect(layers.Layer):
    def __init__(self, drop_connect_rate=0.0, **kwargs):
        super().__init__(**kwargs)
        self.drop_connect_rate = drop_connect_rate

    def call(self, inputs, training=None):
        if training:
            keep_prob = 1.0 - self.drop_connect_rate
            batch_size = tf.shape(inputs)[0]
            random_tensor = keep_prob
            random_tensor += K.random_uniform([batch_size, 1, 1, 1], dtype=inputs.dtype)
            binary_tensor = tf.floor(random_tensor)
            output = tf.math.divide(inputs, keep_prob) * binary_tensor
            return output
        else:
            return inputs

def SEBlock(filters,reduced_filters):
    def _block(inputs):
        x = layers.GlobalAveragePooling2D()(inputs)
        x = layers.Reshape((1,1,x.shape[1]))(x)
        x = layers.Conv2D(reduced_filters, 1, 1)(x)
        x = tfa.activations.mish(x)
        x = layers.Conv2D(filters, 1, 1)(x)
        x = layers.Activation('sigmoid')(x)
        x = layers.Multiply()([x, inputs])
        return x
    return _block

def MBConvBlock(x,kernel_size, strides,drop_connect_rate,output_channels,MBConvBlock_1_True=False):
    output_channels = round_filters(output_channels,width_coefficient,depth_divisor)
    if MBConvBlock_1_True:
        block = layers.DepthwiseConv2D(kernel_size, strides,padding='same', use_bias=False)(x)
        block = layers.BatchNormalization()(block)
        block = tfa.activations.mish(block)
        block = SEBlock(x.shape[3],x.shape[3]/se_ratio)(block)
        block = layers.Conv2D(output_channels, (1,1), padding='same', use_bias=False)(block)
        block = layers.BatchNormalization()(block)
        return block

    channels = x.shape[3]
    expand_channels = channels * expand_ratio
    block = layers.Conv2D(expand_channels, (1,1), padding='same', use_bias=False)(x)
    block = layers.BatchNormalization()(block)
    block = tfa.activations.mish(block)
    block = layers.DepthwiseConv2D(kernel_size, strides,padding='same', use_bias=False)(block)
    block = layers.BatchNormalization()(block)
    block = tfa.activations.mish(block)
    block = SEBlock(expand_channels,channels/se_ratio)(block)
    block = layers.Conv2D(output_channels, (1,1), padding='same', use_bias=False)(block)
    block = layers.BatchNormalization()(block)
    if x.shape[3] == output_channels:
        block = DropConnect(drop_connect_rate)(block)
        block = layers.Add()([block, x])
    return block

def EffNet(num_classes):
    x_input = layers.Input(shape=(default_resolution,default_resolution,input_channels))    
    x = layers.Conv2D(round_filters(32, width_coefficient, depth_divisor), (3,3), 2,padding='same', use_bias=False)(x_input)
    x = layers.BatchNormalization()(x)
    x = tfa.activations.mish(x)
    num_blocks_total = sum(num_repeat)
    block_num = 0
    for i in range(len(kernel_size)):
        round_num_repeat = round_repeats(num_repeat[i], depth_coefficient)
        drop_rate = drop_connect_rate * float(block_num) / num_blocks_total
        x = MBConvBlock(x,kernel_size[i],strides[i],drop_rate,output_filters[i],MBConvBlock_1_True = MBConvBlock_1_True[i])
        block_num += 1
        if round_num_repeat > 1:
            for bidx in range(round_num_repeat - 1):
                drop_rate = drop_connect_rate * float(block_num) / num_blocks_total
                x = MBConvBlock(x,kernel_size[i],1,drop_rate,output_filters[i],MBConvBlock_1_True = MBConvBlock_1_True[i])
                block_num += 1
    x = layers.Conv2D(round_filters(1280, width_coefficient, depth_divisor), 1,padding='same',use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = tfa.activations.mish(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(1,activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=x_input, outputs=x)
    return model

In [21]:
from tensorflow.keras.callbacks import EarlyStopping

# Define and compile the model
model = EffNet(num_classes=len(styles))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define EarlyStopping callback
early_stopper = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    batch_size=8,
    epochs=10,
    validation_data=(X_val, y_val),
    callbacks=[early_stopper],
    verbose=1  
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

Epoch 1/10


NameError: in user code:

    File "/home/myeong/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/myeong/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/myeong/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/myeong/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/home/myeong/.local/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filenl1jb1ae.py", line 47, in tf__call
        ag__.if_stmt(ag__.ld(training), if_body,